# 아티스트 추천 시스템 만들기
---

#### 콘텐츠 추천 알고리즘의 진화
1. 고전적 추천 시스템
* 협업 필터링 : 대규모의 사용자 행동 정보를 분석해서 비슷한 성향의 사람들이 좋아하는 항목을 추천.<br>
(예를 들어, '라면'을 구입한 사용자가 '생수'를 많이 구입한다면, 라면을 구매하는 사람에게 생수를 추천한다.)
> * 비슷한 패턴을 가진 사용자나 항목을 추출하기 위해 **'행렬분해(Matrix Factorization)', k-최근접 이웃 알고리즘(k-Nearest Neighbor algorithm:KNN)**을 주로 사용함.<br>
> * 다만 여러 문제가 있는데, 
> * 1. **콜드 스타트(Cold Start)**라고 일컬어지는 문제를 예로 들면 기존에 없던 새로운 항목이 추가되면 추천 시스템이 작동하기 어려움<br>
 (음악 시스템의 경우 신곡이 발표가 됐다면 추천할 수 있는 <U>정보가 쌓이기 전까지 추천이 어려움</U>)
> * 2. 행렬분해는 사용자 수가 커짐에 따라 계산이 몇 시간에서 며칠까지 걸리기도 함.
> * 3. 사람들은 소수의 인기 항목에 주로 관심을 보이기 때문에, 관심이 적은 항목은 추천에서 벗어나게 됨.
* 콘텐츠 기반 필터링 : 협업 필터링이 사용자의 행동 기록을 분석한다면, 콘텐츠 기반 필터링은 항목 자체를 분석.<br>
(예를 들어, 신곡의 장르, 비트, 음색 등 몇 백 개의 특성을 추출한 뒤, 비슷한 프로파일의 곡을 추천함)
> * 아이템 분석 알고리즘이 핵심이며, **군집 분석, 신경망 네트워크, tf-idf(term frequency-inverse document frequency)** 등의 기술을 사용함
> * 문제점은
> * 1. 다양한 형식의 항목을 추천하기 어려움
> * 2. 여러 형식의 항목을 추천하기 어려움<br>
(예를 들어 음악과 사진, 비디오를 동시에 추천하는 경우, 프로파일을 구성하기 매우 어려워짐)<br><br>

2. 모델 기반 협력 필터링<br>
협력 필터링을 고도화한 것으로, 자료 내 내재된 패턴을 이용하는 알고리즘<br>
(예를 들어, <오징어 게임>을 좋아하는 경우, 이 정보를 그대로 사용하지 않고 주위의 정보와 함께 유추한다. 그 사용자는 프로그램을 주연배우 때문에 좋아할 수도 있고, OST나 장르 때문일 수도 있기 때문.)


---


## 1. 데이터 살펴보기

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/Aiffel_Project/Dataset/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 컬럼명이 없으므로 임의로 지정
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있음.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


#### 관심사가 아닌 컬럼 삭제

In [2]:
# 사용하는 컬럼만 남기기.

using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


#### 대문자를 모두 소문자로 변경

In [3]:
# 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 변경

data['artist'] = data['artist'].str.lower()

data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
# 확인차 첫 번째 유저가 어떤 아티스트의 음악을 듣는지 확인

condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


#### 우선 확인할 항목
* 유저 수, 아티스트 수, 인기 많은 아티스트
* 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
* 유저 play 횟수 중앙값에 대한 통계

In [5]:
# 유저 수

data['user_id'].nunique()

358868

In [6]:
# 아티스트 수

data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트

artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계

user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계

user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

#### 검증을 위한 유저 정보 만들기

In [10]:
# 좋아하는 아티스트 목록
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가. 

data.tail(10)       # 잘 추가되었는지 확인.

/var/folders/p3/fjkwnjwd4dzcfcqt47dlgrm40000gn/T/ipykernel_3354/2046406219.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가.


,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


## 2. 데이터 전처리

### 2-1. indexing
user와 artist에 indexing을 진행<br>
> pandas.DataFrame.unique(): 특정 컬럼에 포함된 유니크한 데이터만 모아줌.

In [12]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드(idx는 index의 약자)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인

print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 함. 
print(artist_to_idx['black eyed peas'])

358868
376


In [14]:
# indexing을 통해 데이터 컬럼 내부 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구할 것. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


---


### 추천 피드백 방법
현재 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇 번 플레이했나 하는 것뿐.<br>
따라서 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백으로 이를 판단할 것.<br>
(플레이 횟수, 플레이 시간 이외에도 클릭 수, 구매 여부, 플레이 스킵 여부, 검색 기록, 방문 페이지 이력, 구매 내역, 심지어 마우스 움직임 기록까지 활용 가능) => 도메인 지식의 필요성
* 암묵적 피드백 데이터셋의 특징
>* 부정적인 피드백이 없다.(No Negative Feedback)
>* 애초에 잡음이 많다.(Inherently Noisy)
>* 수치는 신뢰도를 의미한다.(The numerical value of implicit feedback indicates confidence)
>* Implicit-feedback Recommender System의 평가는 적절한 방법을 고민해봐야 한다.(Evaluation of implicit-feedback recommender requires appropriate measures)

In [15]:
# 1회만 play한 데이터의 비율을 보는 코드

only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


#### 현재 추천 시스템에선 다음 규칙을 가정할 것.
> * 한 번이라도 들었으면 선호한다고 판단한다.
> * 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.


---

## 3. model 생성 : 협업 필터링
### 3-0. 행렬 분해
m명의 사용자가 n명의 아티스트에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 평가 행렬(Rating Matrix)를 만들 것.<br>
이후 행렬에 Matrix Factorization(MF, 행렬 분해) 모델을 사용할 것.
![matrix factorization](matrix_factorization.png)
* (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면, R을 P와 Q의 행렬곱으로 표현 가능할 수 있다는 아이디어.<br>
* k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리함.

#### 적용할 분해 방식
MF 모델은 평가 행렬 R을 두 개의 Feature Matrix P와 Q로 분해함. 그렇다면 어떤 Feature로 분해?
* 1. (m, k): Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬. 즉 **사용자의 특성(Feature) 벡터**
* 2. (n, k): Feature Matrix Q는 **영화의 특성 벡터**<br>


> 예를 들어, P의 첫 번째 벡터 $P_{0}$와 Q의 첫 번째 벡터 $Q_{0}$을 내적하면 $R_{0,0}$ 값이 나오게 됨. => 즉 사용자의 영화 선호도를 알 수 있다.


#### 벡터를 잘 만드는 기준은 **유저 i의 벡터($U_{i}$)와 아이템 j의 벡터($I_{j}$)를 내적**했을 때, **유저 i가 아이템 j에 대해 평가한 수치($M_{ij}$)와 비슷**한지 여부이다.


* 이전 스탭에서 black eyed peas'를 play했다는 데이터를 추가했기 때문에, 내가 추가한 사용자의 벡터와 black eyed peas의 벡터를 곱했을 때의 값이 1에 가깝게 나오도록 모델이 잘 학습하는 것을 목표로 함.<br>
* 마찬가지로 다른 아티스트인 queen에 대해서도 이 사용자의 벡터와 queen의 벡터를 곱하여 수치를 예상할 수 있음.

### 3-1. CSR Matrix 적용
* 다만 유저는 36만 명이고 아티스트는 29만 명이므로, 행렬의 각 원소에 정수 한 개 (1byte)만 들어가도 36만 * 29만 * 1byte로 약 97GB나 필요함.
* 따라서 메모리를 덜 사용할 수 있도록 **CSR(Compressed Sparse Row) Matrix**를 사용할 것(0이 아닌 유효한 데이터만 메모리에 사용)

In [16]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

### 3-2. Matrix Factorization 모델을 implicit 패키지를 사용하여 학습
implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지.<br>
(패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용할 것)<br>
> Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적임.


In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습 내용과는 무관.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

ModuleNotFoundError: No module named 'implicit'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언


als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해.)


csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
# 모델 훈련

als_model.fit(csr_data_transpose)

---


## 4. 결과 확인
* Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
* 두 벡터를 곱하면 어떤 값이 나오는지

In [ ]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [ ]:
zimin_vector

In [ ]:
black_eyed_peas_vector

In [ ]:
# zimin과 black_eyed_peas를 내적.

np.dot(zimin_vector, black_eyed_peas_vector)

1에서 한참 낮은 수치 약 0.49가 나옴. factors를 늘리거나 iterations를 늘려야 함.

* queen에 대한 선호도를 예측

In [ ]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

하지만 결과값이 나와도 객관적인 지표를 정하기 어렵기 때문에, 데이터나 결과물에 대한 이해도가 높아야 한다.

---


## 5. 비슷한 아티스트 찾기
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾기.<br>
coldplay를 예시로 적용

In [ ]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

(아티스트의 id, 유사도) Tuple 로 반환함. <br><br>


#### 아티스트의 id를 다시 아티스트의 이름으로 매핑.

In [ ]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

#### 위 과정을 반복하는 함수 생성

In [ ]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

#### 여러 아티스트로 확인.

In [ ]:
get_similar_artist('2pac')

In [ ]:
get_similar_artist('lady gaga')

---


## 6. 유저에게 아티스트 추천하기
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 좋아할 만한 아티스트를 추천 받을 것.<br>
filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument.

In [ ]:
user = user_to_idx['zimin']

# recommend에서는 user*item CSR Matrix를 받음
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

In [ ]:
[idx_to_artist[i[0]] for i in artist_recommended]

rihanna를 추천함.<br><br>

#### AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용해, zimin 사용자가 이 추천에 기여한 정도를 확인.

In [ ]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method는 추천한 콘텐츠의 '점수에 기여한 다른 콘텐츠'와, '기여도'(합이 콘텐츠의 점수)를 반환

In [ ]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

beyoncé와 black eyed peas가 가장 크게 기여한 것을 알 수 있음.<br>
+ filter_already_liked_items=True로 했는데 앞에 maroon 5이 나오는 것은 implicit 버전 0.4.2에서 발생하는 버그.